In [1]:
import math
from skimage import io, color
from skimage.transform import resize
import numpy as np

In [2]:
class SuperPixels(object):

    def __init__(self, h, w, l=0, a=0, b=0):
        self.update(h, w, l, a, b)
        self.pixels = []

    def update(self, h, w, l, a, b):
        self.h = h
        self.w = w
        self.l = l
        self.a = a
        self.b = b

In [3]:
def atribuir_superPixel(h,w, img):
    return SuperPixels(h, w,img[h,w][0],img[h,w][1],img[h,w][2])

def lab2rgb(path, lab_arr):
    rgb_arr = color.lab2rgb(lab_arr)
    im = (rgb_arr * 255).astype(np.uint8)
    io.imsave(path, im)

## Função para inicializar cada centro do cluster
Para saber a posição h,w do pixel central: Sabendo que temos S que é a distância entre os centros, se assumir como quadrados e de vizinhos, o valor da distância do pixel central será S / 2.

In [4]:
def inicializar_cluster_centro(S, img, img_h, img_w, clusters):
    h = S // 2
    w = S // 2
    while h < img_h:
        while w < img_w:
            clusters.append(atribuir_superPixel(h, w,img))
            w += S
        w = S // 2
        h += S
    return clusters

In [5]:
image = io.imread("picos.jpg")
img = resize(image, (400,400),anti_aliasing=True)
img = color.rgb2lab(img)

k = 100
m = 20

img_h = img.shape[0] # Image Height
img_w = img.shape[1] # Image Width

N = img_h * img_w  # Total number of pixels in the image
S = int(math.sqrt(N /k)) # average size of each superpixel

clusters = []
inicializar_cluster_centro(S, img, img_h, img_w, clusters)
for c in clusters:
    print(c.h, c.w)

20 20
20 60
20 100
20 140
20 180
20 220
20 260
20 300
20 340
20 380
60 20
60 60
60 100
60 140
60 180
60 220
60 260
60 300
60 340
60 380
100 20
100 60
100 100
100 140
100 180
100 220
100 260
100 300
100 340
100 380
140 20
140 60
140 100
140 140
140 180
140 220
140 260
140 300
140 340
140 380
180 20
180 60
180 100
180 140
180 180
180 220
180 260
180 300
180 340
180 380
220 20
220 60
220 100
220 140
220 180
220 220
220 260
220 300
220 340
220 380
260 20
260 60
260 100
260 140
260 180
260 220
260 260
260 300
260 340
260 380
300 20
300 60
300 100
300 140
300 180
300 220
300 260
300 300
300 340
300 380
340 20
340 60
340 100
340 140
340 180
340 220
340 260
340 300
340 340
340 380
380 20
380 60
380 100
380 140
380 180
380 220
380 260
380 300
380 340
380 380


## Associar cada pixel ao centro do cluster mais próximo
Para isso utilizou-se o calculo do gradiente

In [6]:
def calculo_gradiente(h, w,img,img_w,img_h):
    if w + 1 >= img_w:
        w = img_w - 2
    if h + 1 >= img_h:
        h = img_h - 2
    grad = img[w + 1, h + 1][0] - img[w, h][0] + img[w + 1, h + 1][1] - img[w, h][1] + img[w + 1, h + 1][2] - img[w, h][2]
    return grad

Reatribui o centro do cluster ao pixel com menor gradiente

In [7]:
def reatribuir_cluster_centro_gradiente(clusters,img):
    for c in clusters:
        cluster_gradient = calculo_gradiente(c.h, c.w,img,img_w,img_h)
        for dh in range(-1, 2):
            for dw in range(-1, 2):
                H = c.h + dh
                W = c.w + dw
                new_gradient = calculo_gradiente(H,W, img,img_w,img_h)
                if new_gradient < cluster_gradient:
                    c.update(H, W,img[H,W][0], img[H,W][1],img[H,W][2])

Atribui os pixels ao cluster mais próximo

In [8]:
def associar_pixel_ao_cluster(clusters,S,img,img_h,img_w,tag,dis):
    for c in clusters:
        for h in range(c.h - 2 * S, c.h + 2 * S):
            if h < 0 or h >= img_h: 
                continue
            for w in range(c.w - 2 * S, c.w + 2 * S):
                if w < 0 or w >= img_w: 
                    continue
                l, a, b = img[h,w]
                Dc = math.sqrt(math.pow(l - c.l, 2) + math.pow(a - c.a, 2) + math.pow(b - c.b, 2))
                Ds = math.sqrt(math.pow(h - c.h, 2) + math.pow(w - c.w, 2))
                D = math.sqrt(math.pow(Dc / m, 2) + math.pow(Ds /S, 2))
                if D < dis[h,w]: # Dis é o vetor de distância (inicializado sendo os valores inf)
                    if (h, w) not in tag:
                        tag[(h, w)] = c
                        c.pixels.append((h, w))
                    else:
                        tag[(h, w)].pixels.remove((h, w))
                        tag[(h, w)] = c
                        c.pixels.append((h, w))
                    dis[h, w] = D

In [9]:
def atualizar_cluster_media(clusters):
    for c in clusters:
        sum_h = sum_w = number = 0
        #print("c.pixels",c.pixels)
        for p in c.pixels:
            sum_h += p[0]
            sum_w += p[1]
            number += 1
            H = sum_h // number
            W = sum_w // number
            c.update(H, W,img[H, W][0], img[H, W][1], img[H, W][2])

In [10]:
def avg_color_cluster(img,name,clusters):
    image = np.copy(img)
    for c in clusters:
        for p in c.pixels:
            image[p[0],p[1]][0] = c.l
            image[p[0],p[1]][1] = c.a
            image[p[0],p[1]][2] = c.b
        # To change the color of cluster center to Black
        image[c.h, c.w][0] = 0
        image[c.h, c.w][1] = 0
        image[c.h, c.w][2] = 0
    lab2rgb(name, image)

In [11]:
def slic(S,img,img_h,img_w,clusters,tag,dis):
    clusters = inicializar_cluster_centro(S,img,img_h,img_w,clusters)
    reatribuir_cluster_centro_gradiente(clusters,img)
    
    for i in range(10):
        associar_pixel_ao_cluster(clusters,S,img,img_h,img_w,tag,dis)
        atualizar_cluster_media(clusters)
        
    name = 'out_m{m}_k{k}.jpg'.format(loop=i, m=m, k=k)
    avg_color_cluster(img, name, clusters)
    return clusters

In [15]:
image = io.imread("picos.jpg")

img = resize(image, (500,500),anti_aliasing=True)
img = color.rgb2lab(img)

k = 100
m = 20

img_h = img.shape[0]
img_w = img.shape[1]

N = img_h * img_w
S = int(math.sqrt(N /k))

clusters = []
tag = {}
dis = np.full((img_h, img_w), np.inf)

In [16]:
cluster = slic(S,img,img_h,img_w,clusters,tag,dis)